In [ ]:
# packages to install
# pip install dnspython
# pip install 'pymongo[srv]'

In [1]:
# make sure that dnspython pymongo are installed in your computer
# import sqlite3
import sqlite3
conn = sqlite3.connect('db2.db')
print("connection successfull")

connection successfull


In [ ]:

# Below are the QUERIES to create Tables in database

""" 
Artist Table  

"""

# aID	Name	Birthdate	DeathDate	commission	street	city	state	zipcode

conn.execute('''CREATE TABLE ARTIST
 (AID INT PRIMARY KEY     NOT NULL,
 NAME           TEXT    NOT NULL,
 BIRTHDATE      TEXT    NOT NULL,
 DEATHDATE      TEXT    ,
 COMMISSION   INT NOT NULL,
 STREET TEXT NOT NULL,
 CITY TEXT NOT NULL,
 STATE TEXT NOT NULL,
 ZIPCODE TEXT NOT NULL)''')


""" 
Artwork Table  

"""

# aID	artID	title	creationDate	acquisitionDate	price	form

conn.execute('''CREATE TABLE ARTWORK
 (AID INT      NOT NULL,
  ARTID INT NOT NULL,
  TITLE TEXT NOT NULL,
  CREATIONDATE      TEXT    NOT NULL,
  ACQUISITIONDATE      TEXT    NOT NULL,
  PRICE INT NOT NULL,
  FORM TEXT NOT NULL,
  PRIMARY KEY (ARTID)
  )''')

""" 
Bought Table  

"""

# artID	cid	saleDate

conn.execute('''CREATE TABLE BOUGHT
(ARTID INT NOT NULL,
  CID INT NOT NULL,
  SALEDATE TEXT NOT NULL,
  PRIMARY KEY (CID)
  )''')


""" 
CUSTOMER Table  

"""

# CID	NAME	STREET	CITY	STATE	ZIPCODE	


conn.execute('''CREATE TABLE CUSTOMER
(CID INT NOT NULL,
 NAME TEXT NOT NULL,
 STREET TEXT NOT NULL,
 CITY TEXT NOT NULL,
 STATE TEXT NOT NULL,
 ZIPCODE TEXT NOT NULL,
 PRIMARY KEY (CID)
  )''')

""" 
STATE Table  

"""

# STATEAB	STATENAME


conn.execute('''CREATE TABLE STATE
 (STATEAB TEXT NOT NULL,
  STATENAME TEXT NOT NULL,
   PRIMARY KEY (STATEAB)
  )''')




In [2]:
# code for populating tables from the csv files

import csv
def populateTable(filename,tablename,columns):
    with open(filename, 'r') as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        rows = []
        # extracting field names through first row
        fields = next(csvreader)
        brackets = ",".join(['"{}"']*len(columns)) 
        for row in csvreader:
            rows.append(row)
            conn.execute((f' INSERT INTO {tablename} ({",".join(columns)}) VALUES ({brackets})').format(*row))

# code for Displaying Table

def DisplayTable(tablename,length=100):
    cursor = conn.execute(f'select * from {tablename} LIMIT {length}')
    for i in cursor:
        print(i)

In [3]:
populateTable("ARTIST.csv","ARTIST",["aID","NAME","BIRTHDATE","DEATHDATE",\
                                     "COMMISSION","STREET","CITY","STATE","ZIPCODE"])
populateTable("ARTWORK.csv","ARTWORK",["AID","ARTID","TITLE","CREATIONDATE","ACQUISITIONDATE","PRICE","FORM"])

populateTable("BOUGHT.csv","BOUGHT",["ARTID","CID","SALEDATE"])

populateTable("CUSTOMER.csv","CUSTOMER",["CID","NAME","STREET","CITY","STATE","ZIPCODE"])

populateTable("STATE.csv","STATE",["STATEAB","STATENAME"])

print("Done populating")


Done populating


In [4]:
DisplayTable("ARTIST",5)
DisplayTable("ARTWORK",3)
DisplayTable("BOUGHT",3)
DisplayTable("CUSTOMER",3)
DisplayTable("STATE",3)

(1, 'Abed Abdi', '2000-01-01', 'NULL', 10, '309 Hcounty Road', 'Abbott', 'TX', '76621-0057')
(2, 'Ismail Gulg', '1990-01-03', 'NULL', 11, '405 E Mesquite Street', 'Abbott', 'TX', '76621-0057')
(3, 'Shakir Ali', '1995-02-04', 'NULL', 12, '5000 Spectrum Street', 'Addison', 'TX', '75001-6880')
(4, 'Abdur Rahman', '1970-02-05', 'NULL', 10, '1000 County Road', 'Bradley', 'Ok', '73011-0121')
(5, 'Kalipada Ghoshal', '1994-01-01', 'NULL', 10, 'Boundary Road', 'Bradley', 'OK', '73011-9600')
(1, 1, 'Refugee Camp', '2020-09-09', '2021-02-07', 12000, 'painting')
(1, 2, 'Waiting', '2020-12-28', '2021-02-07', 15000, 'painting')
(1, 3, 'Palestenian Women', '2021-01-01', '2021-02-07', 1700, 'painting')
(1, 1, '2021-07-08')
(1, 2, '2021-07-08')
(3, 3, '2021-08-08')
(1, 'Mahbod Seraji ', '1500 E Exchange Road', 'Allen', 'TX', '75002-4504')
(2, 'Shazadi Seraji', '1500 E Exchange Road', 'Allen', 'TX', '75002-4504')
(3, 'Nadia Hashimi', '1405 Julian Street', 'Addison', 'TX', '75001-4633')
('TX', 'Texas')
(

In [5]:
import json
# function for getting data for populating artist collection on mongodb

def getArtistCollection():
    cursor = conn.execute("""SELECT ARTIST.AID,ARTIST.NAME,ARTIST.BIRTHDATE,
    STATE.STATENAME,ARTWORK.TITLE,ARTWORK.PRICE,ARTWORK.FORM
    FROM ARTIST
    LEFT JOIN ARTWORK ON ARTIST.AID = ARTWORK.AID
    LEFT JOIN STATE ON LOWER(STATE.STATEAB)= LOWER(ARTIST.STATE);""")

    artists = {}
    artworks = {}
    docs = []
    for row in cursor:

        if row[1] not in artworks:
            artworks[row[1]] = [{"title":row[4],"price":row[5],"form":row[6]}]
            artists[row[1]] = {"_id":row[0],"name":row[1],"birthdate":row[2],'state':row[3]}
        else:
            artworks[row[1]].append({"title":row[4],"price":row[5],"form":row[6]})
    for artist in artists:
        temp = dict(artists[artist]) 
        temp["artworks"] = artworks[artist]
        docs.append(temp)
    return docs

with open("Artist.json", "w") as outfile:
    json.dump(getArtistCollection(), outfile)
print("done")
        

done


In [6]:
# function for getting data for  populating ARTWork collection on mongodb

def getArtWorkCollection():
    cursor = conn.execute("""SELECT ARTWORK.ARTID,  ARTWORK.TITLE, ARTWORK.CREATIONDATE,ARTWORK.PRICE,
    ARTWORK.FORM,ARTIST.NAME,CUSTOMER.NAME, CUSTOMER.CITY,STATE.STATENAME
    FROM ((((Artwork left join bought on artwork.artid = bought.artid)
    left join artist on artwork.aid = artist.aid)
    left join customer on customer.cid = bought.cid)
    left join state on LOWER(customer.state) = LOWER(state.stateab))
    """)
    artworks = {}
    customers = {}
    docs=[]
    for row in cursor:
        if row[1] not in artworks:
            if row[6] is None:
                customers[row[1]] = []
            else:
                customers[row[1]] = [{"name":row[6],"city":row[7],"state":row[8]}]
            artworks[row[1]] = {"_id":row[0],"name":row[1],"creationdate":row[2],'price':row[3],'form':row[4],"artistname":row[5]}
        else:
            customers[row[1]].append({"name":row[6],"city":row[7],"state":row[8]})
    for artwork in artworks:
        temp = dict(artworks[artwork]) 
        temp["customers"] = customers[artwork]
        docs.append(temp)    
    return docs

with open("Artwork.json", "w") as outfile:
    json.dump(getArtWorkCollection(), outfile)
print("done")

done


In [7]:
# uncomment the lines below to connect to local client

# import pymongo
# myclient = pymongo.MongoClient("mongodb://localhost:27017/")

import urllib 
from pymongo import MongoClient

username = urllib.parse.quote_plus('yashdemouser')
password = urllib.parse.quote_plus('Zb6cLOXkgKmer9PX')
connectionstring = 'mongodb+srv://%s:%s@cluster0.gltm7.mongodb.net/?retryWrites=true&w=majority' % (username, password)

myclient = MongoClient(connectionstring)


mydb = myclient["db2"]

print("connection successfull")

connection successfull


In [ ]:
# this cell is for inserting all collections from json file
# use only once or else you will get duplicate insertion error
mycol = mydb["ARTIST"]

data = []
with open('Artist.json') as json_file:
    data = json.load(json_file)
    
mycol.insert_many(data)

mycol = mydb["ARTWORK"]

with open('Artwork.json') as json_file:
    data = json.load(json_file)
mycol.insert_many(data)

print("Done Inserting all collections ......")


In [ ]:
#This code for deleting all both collections
# use only when required

mycol = mydb["ARTIST"]
result = mycol.delete_many({})
print(result.deleted_count, " documents deleted.")

mycol = mydb["ARTWORK"]
result = mycol.delete_many({})
print(result.deleted_count, " documents deleted.")


In [ ]:
# queries for dropping tables
conn.execute(''' DROP TABLE ARTIST ''')
conn.execute(''' DROP TABLE ARTWORK ''')
conn.execute(''' DROP TABLE BOUGHT ''')
conn.execute(''' DROP TABLE CUSTOMER  ''')
conn.execute(''' DROP TABLE STATE''')